In [2]:
# Load API keys from environment variables (recommended for security)
# Removing the API tokens from ipynb file for security

CLOUDFLARE_API_KEY = os.getenv("CLOUDFLARE_API_KEY", "Your Token")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY", "Your Token")
SPOONACULAR_API_KEY = os.getenv("SPOONACULAR_API_KEY", "Your Token")

# Cloudflare API URLs
CLOUDFLARE_VERIFY_URL = "https://api.cloudflare.com/client/v4/user/tokens/verify"
CLOUDFLARE_AI_URL = "https://api.cloudflare.com/client/v4/accounts/32c39b1f7ff91c5ebec63fedf7342109/ai/run/@cf/meta/llama-3-8b-instruct"

# User input for location
#CITY = input("Enter your city: ")
CITY = "San Francisco"

def get_user_city():
    city = input("Enter your city: ").strip()
    if not city:
        print("❌ Invalid city name. Using default: San Francisco.")
        return "San Francisco"
    return city


# Verify Cloudflare API Token
def verify_cloudflare_token():
    headers = {
        "Authorization": f"Bearer {CLOUDFLARE_API_KEY}",
        "Content-Type": "application/json"
    }
    response = requests.get(CLOUDFLARE_VERIFY_URL, headers=headers)

    if response.status_code == 200:
        print("✅ Cloudflare API Token Verified!")
        return True
    else:
        print(f"❌ Cloudflare API Token Verification Failed: {response.status_code}")
        return False

# Fetch weather data
def fetch_weather_data(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather data: {response.status_code}")
        return None

# Get food type based on weather condition
def get_food_type(weather_desc):
    weather_desc = weather_desc.lower()
    if "rain" in weather_desc:
        return "soup"
    elif "clear" in weather_desc or "sun" in weather_desc:
        return "salad"
    elif "cloud" in weather_desc:
        return "stew"
    elif "snow" in weather_desc:
        return "warm drink"
    else:
        return "normal food"

# Get workout recommendation based on weather condition
def get_workout_recommendation(weather_desc):
    weather_desc = weather_desc.lower()
    if "rain" in weather_desc or "storm" in weather_desc:
        return "Indoor workout: Yoga or home exercises"
    elif "clear" in weather_desc or "sun" in weather_desc:
        return "Outdoor activity: Running or cycling"
    elif "snow" in weather_desc:
        return "Indoor workout: Strength training"
    else:
        return "General workout: Mixed cardio and strength"

# Fetch recipe from Spoonacular
recipe_cache = {}

def fetch_recipe(food_type):
    if food_type in recipe_cache:
        return random.choice(recipe_cache[food_type])

    url = f"https://api.spoonacular.com/recipes/complexSearch?query={food_type}&apiKey={SPOONACULAR_API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        recipes = response.json()
        if recipes['results']:
            recipe_cache[food_type] = recipes['results']  # Store all recipes in cache
            return random.choice(recipes['results'])
    print(f"❌ Error fetching recipes: {response.status_code}")
    return None

# Fetch detailed recipe information
def fetch_recipe_details(recipe_id):
    url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={SPOONACULAR_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching recipe details: {response.status_code}")
    return None

def generate_text_with_cloudflare(prompt):
    headers = {
        "Authorization": f"Bearer {CLOUDFLARE_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "messages": [
            {"role": "system", "content": "You are an AI assistant that suggests meals and workouts based on weather."},
            {"role": "user", "content": prompt}
        ]
    }

    response = requests.post(CLOUDFLARE_AI_URL, json=payload, headers=headers)
    #print(response.json())
    if response.status_code == 200:
        return response.json()["result"]
    else:
        print(f"❌ Error with Cloudflare AI: {response.status_code}")
        return "No AI response available"

# Main function to generate recommendations
def generate_recommendations(weather_data):
    if not weather_data:
        return

    for i, forecast in enumerate(weather_data["list"][:24:8]):  # Limit to 3 days
        #if i >= 3:
         #   break

        weather_desc = forecast["weather"][0]["description"]
        date = forecast["dt_txt"]

        # Get food type and workout recommendation
        food_type = get_food_type(weather_desc)
        workout = get_workout_recommendation(weather_desc)

        # Fetch recipe from Spoonacular
        recipe = fetch_recipe(food_type)
        recipe_title = "No Recipe Found"
        recipe_url = "N/A"

        if recipe:
            recipe_details = fetch_recipe_details(recipe['id'])
            if recipe_details:
                recipe_title = recipe_details.get('title', 'Unknown Recipe')
                recipe_url = recipe_details.get('sourceUrl', 'No URL available')

        # Generate AI recommendation with Cloudflare
        prompt = f"The weather on {date} in {CITY} is {weather_desc}. Suggest a healthy meal and an exercise routine based on this weather."
        ai_response = generate_text_with_cloudflare(prompt)

        # Print recommendations
        print(f"📅 Date: {date}")
        print(f"🌤 Weather: {weather_desc}")
        print(f"🍽 Recommended Food: {recipe_title} ({recipe_url})")
        print(f"💪 Workout Recommendation: {workout}")
        print(f"🤖 AI Suggestion: {ai_response}\n")

# Run the program
if __name__ == "__main__":
    if verify_cloudflare_token():
        CITY = get_user_city()
        weather_data = fetch_weather_data(CITY)
        generate_recommendations(weather_data)



✅ Cloudflare API Token Verified!
Enter your city: MIAMI
📅 Date: 2025-02-05 00:00:00
🌤 Weather: few clouds
🍽 Recommended Food: 10-Minute Bean Stew (https://www.foodista.com/recipe/M57WVNMK/10-minute-bean-stew)
💪 Workout Recommendation: General workout: Mixed cardio and strength
🤖 AI Suggestion: {'response': 'Perfect Miami weather! With a clear sky and a calm atmosphere, I\'d recommend a meal and workout routine that\'s refreshing, revitalizing, and perfect for a wonderful Tuesday morning:\n\n**Breakfast Suggestion:**\n\nTo start your day with a burst of energy, I suggest a "Miami Mango Sunrise" bowl:\n\n* 1 cup cooked quinoa\n* 1/2 cup diced fresh mango\n* 1/2 cup mixed berries (blueberries, strawberries, raspberries)\n* 1/4 cup chopped almonds\n* 1 tablespoon honey\n* 1 tablespoon freshly squeezed orange juice\n\nCombine all the ingredients in a bowl and enjoy the sweetness of the mango and berries, balanced by the crunch of the almonds and the quinoa. The perfect breakfast to fuel you